In [1]:
import sys
from pathlib import Path

# Get the absolute path of the current notebook
notebook_path = Path().resolve()

# Get the project root directory (which is the parent of the 'notebooks' directory)
project_root = notebook_path.parent

# Add BOTH the project root and the src directory to the Python path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
if str(project_root / 'src') not in sys.path:
    sys.path.append(str(project_root / 'src'))

# Now, we can import our modules
from src.data_handling import DataHandler
from src.mlp_model import MLPModel

Using MPS device (Apple Silicon GPU)


#### Initialize DataHandler
This cell creates an instance of the `DataHandler` class. It will load the dataset, define the feature/target sets, and pre-fit the necessary data processing objects. The test year is set to 2020.

In [2]:
dh = DataHandler(test_year=2020)

DataHandler initialized - Using 52 features - Test year: 2020


### **Model: MLP with 3 Hidden Layers**

#### Initialize MLPModel Handler
Here, we create an instance of the `MLPModel` class, explicitly setting `depth=3`. The `model_name` ensures all artifacts for this architecture are saved to unique files.

In [3]:
mlp_depth3 = MLPModel(dh, model_name='mlp_depth3', depth=3)

MLPModel initialized with model name mlp_depth3 and depth 3.
Optuna study will be stored in  : /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/optuna/mlp_depth3_study.pkl
Trained model will be stored in : /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/models/mlp_depth3_model.pth
Final preds will be stored in   : /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/preds/mlp_depth3_preds.csv


#### Run Optuna Hyperparameter Study (Depth=2)
This cell runs the Optuna study. For this specific 2-layer architecture, it will find the optimal learning rate, weight decay, batch size, and the number of neurons and dropout rates for both hidden layers.

In [4]:
mlp_depth3.run_optuna_study(
    n_trials=50,
    timeout=30,               # Stop study after 30 minutes
    max_epochs=40,            # Max epochs to train each trial
    min_resource=4,          # Min epochs before pruning
    reduction_factor=2,
    use_pca=False
)

[I 2025-10-23 11:00:26,067] A new study created in memory with name: mlp_depth3_study
[I 2025-10-23 11:00:47,411] Trial 8 pruned. 
[I 2025-10-23 11:00:48,551] Trial 4 pruned. 
[I 2025-10-23 11:00:54,872] Trial 1 pruned. 
[I 2025-10-23 11:01:05,472] Trial 3 pruned. 
[I 2025-10-23 11:01:21,704] Trial 0 pruned. 
[I 2025-10-23 11:01:21,877] Trial 11 pruned. 
[I 2025-10-23 11:01:35,544] Trial 10 pruned. 
[I 2025-10-23 11:02:10,253] Trial 15 pruned. 
[I 2025-10-23 11:02:18,952] Trial 16 pruned. 
[I 2025-10-23 11:02:20,371] Trial 7 pruned. 
[I 2025-10-23 11:02:33,103] Trial 19 pruned. 
[I 2025-10-23 11:02:34,304] Trial 12 finished with value: 1.007140616575877 and parameters: {'learning_rate': 0.017362581240268876, 'weight_decay': 0.0004180354987819877, 'batch_size': 256, 'n_hidden_1': 104, 'dropout_rate_1': 0.4568171221117885, 'n_hidden_2': 64, 'dropout_rate_2': 0.31583124018754427, 'n_hidden_3': 16, 'dropout_rate_3': 0.46728165592256327}. Best is trial 12 with value: 1.007140616575877.
[I 2

--------------------
Study concluded and saved to /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/optuna/mlp_depth3_study.pkl.
Best trial: 5
Best loss: 1.0005678335825603
Best params: {'learning_rate': 0.006893765839887174, 'weight_decay': 0.0003526696873415923, 'batch_size': 256, 'n_hidden_1': 56, 'dropout_rate_1': 0.0954602912874219, 'n_hidden_2': 96, 'dropout_rate_2': 0.032160950239147834, 'n_hidden_3': 56, 'dropout_rate_3': 0.24526333680741458}
Best epoch: 26


#### Train Final Model (Depth=3)
Using the best hyperparameters and the optimal number of training epochs found by Optuna, this cell trains the final 3-layer MLP on the entire training dataset. The model's weights (`.pth` file) are saved to the results directory.

In [5]:
mlp_depth3.train_final_model(patience=10) # Stop if loss doesn't improve for 10 epochs

Training mlp_depth3 with depth 3 for 26 epochs with patience of 10 epochs.
Using batch size: 256 and best params: {'learning_rate': 0.006893765839887174, 'weight_decay': 0.0003526696873415923, 'batch_size': 256, 'n_hidden_1': 56, 'dropout_rate_1': 0.0954602912874219, 'n_hidden_2': 96, 'dropout_rate_2': 0.032160950239147834, 'n_hidden_3': 56, 'dropout_rate_3': 0.24526333680741458}.
Epoch    0, Loss: 1.055174
Epoch    1, Loss: 1.005213
Epoch    2, Loss: 1.002300
Epoch    3, Loss: 0.999997
Epoch    4, Loss: 0.999820
Epoch    5, Loss: 0.997259
Epoch    6, Loss: 0.997366
Epoch    7, Loss: 0.997006
Epoch    8, Loss: 0.997374
Epoch    9, Loss: 0.995933
Epoch   10, Loss: 0.994634
Epoch   11, Loss: 0.996028
Epoch   12, Loss: 0.996111
Epoch   13, Loss: 0.993766
Epoch   14, Loss: 0.995062
Epoch   15, Loss: 0.993454
Epoch   16, Loss: 0.994629
Epoch   17, Loss: 0.995063
Epoch   18, Loss: 0.994409
Epoch   19, Loss: 0.993791
Epoch   20, Loss: 0.993231
Epoch   21, Loss: 0.993995
Epoch   22, Loss: 0.99

#### Generate Final Predictions (Depth=3)
Finally, we load the trained 3-layer model and use it to make predictions on the held-out 2020 test set.

In [6]:
# Make predictions on the 2020 test data
preds_depth3 = mlp_depth3.make_final_predictions()
print("\nMLP (Depth=3) Predictions (first 5 rows):")
print(preds_depth3[:5])

mlp_depth3 predictions saved to: /Users/arvindsuresh/Documents/Github/Election-prediction-May-2025/2020-results-20251023/preds/mlp_depth3_preds.csv.

MLP (Depth=3) Predictions (first 5 rows):
[[0.1417415  0.28043145 0.02995418 0.5478728 ]
 [0.14857827 0.29302874 0.03943167 0.51896125]
 [0.17976786 0.21175139 0.01185497 0.5966258 ]
 [0.10410244 0.27115166 0.01873205 0.6060139 ]
 [0.10320204 0.27827454 0.02192812 0.5965952 ]]
